# Training dormain specific Word2Vec word embeddings with Gensim

In [2]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
pd.set_option('max_colwidth', 1000)
pd.options.display.max_rows = 500
# Read file
df = pd.read_pickle('df_orig.pkl')

In [4]:
df.shape

(66806, 2)

In [5]:
df.head()

,Doc_Id,consumer_complaint_narrative
190126,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
190135,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
190155,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
190207,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
190208,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [6]:
df=df.reset_index(drop=True)

In [7]:
df.head()

,Doc_Id,consumer_complaint_narrative
0,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
1,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
2,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
3,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
4,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [8]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
#     txt = [token.lemma_ for token in doc if not token.is_stop]
    txt = [token.lemma_ for token in doc]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [9]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['consumer_complaint_narrative'])

In [10]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 8.91 mins


In [11]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(65552, 1)

In [12]:
df_clean['clean'] = df_clean['clean'].str.replace('x', '')
df_clean['clean'] = df_clean['clean'].str.replace('-PRON-', '')

In [13]:
df_clean.head()

,clean
0,have claim i owe -PRON- for year despite the proof of payment i send -PRON- cancel check and -PRON- ownpaid invoice for -PRON- continue to insist i owe -PRON- and collection agency be after -PRON- how can i stop this harassment for a bill i already pay four year ago
1,due to inconsistency in the amount owe that i be tell by m t bank and the amount that be report to the credit reporting agency i be advise to write a good will letter in order to address the issue and request the negative entry be remove from -PRON- credit report all together i have a vehicle that be steal and -PRON- be declare a total loss by insurance company the insurance company and the gap insurancw companypaid the outstanding balance of the loan but i be tell by m t bank that there be still a balance due on the loan in good faith without have receive any proof as to why there be still a balance i make a partial payment towards the remain debt i then send the goodwill letter still offer to pay the remainder of the debt but in echange for the removal of the negative entry on -PRON- credit report at one point in per -PRON- credit monitoring agency -PRON- show a delinquent balance of but when i check -PRON- credit report again on there be a delinquent balance of the monie from...
2,in -PRON- wage that i earn at -PRON- job decrease by almost half by i know i be in trouble with -PRON- home loan i begin contact wfb whom -PRON- home loan be with for assitance and option in early i begin the loan modification process with well fargo bank i be tell that -PRON- would not assist -PRON- with anything financial on -PRON- home loan until i fall day behind though at the time i start to inquire for assistance from wfb i be only a few week behind so i begin work with a program call -PRON- approve -PRON- for a variety of assistence and reach out to well fargo bank to determine what -PRON- could assist with well fargo then turn down the assistance from and finally offer to do a loan modification for -PRON- the outcome be totally unknow about what i would be offer in the end by wfb for assistance well fargo lose -PRON- paperwork twice during this process so -PRON- take month from the time i start to the time -PRON- paperwork begin to be process for some kind of app...
3,i have an open and current mortgage with chase bank chase be report the loan payment to but be surpresse the information and report the loan as discharge in bk this mortgage be reaffirm in a chapter bk discharge date chase keep refer to bk law for chapter and -PRON- keep provide documentation for chapter and the account should be open and current with all the payment
4,be submit at the time i submit this complaint i have deal with rushmore mortgage directly endeavor to get -PRON- to stop the continuous daily call i be receive try to collect on a mortgage for which i be not responsible due to bankruptcy -PRON- deny have knowledge of the bankruptcy even though i have speak with -PRON- about -PRON- repeatedly and have write -PRON- repeatedly reference the bankruptcy request -PRON- to cease the pursuit -PRON- continue to do so when -PRON- be unable to trick -PRON- into pay force -PRON- into pay in retaliation -PRON- place report to -PRON- credit bureaus a past due mortgage amount that have be discharge in federal court on rushmore respond the referenced complaint indicate that -PRON- would remove the reporting from -PRON- bureau yet -PRON- be still there now in i would like -PRON- to remove -PRON- immediately and send -PRON- a letter indicate that -PRON- should not have be there in the first place and -PRON- be go to remove -PRON- from all ...


In [14]:
from gensim.models.phrases import Phrases, Phraser

C:\Users\alber\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO - 18:47:55: 'pattern' package not found; tag filters are not available for English


In [35]:
sent = [row.split() for row in df_clean['clean']]

In [37]:
sent[1]

['due',
 'to',
 'inconsistency',
 'in',
 'the',
 'amount',
 'owe',
 'that',
 'i',
 'be',
 'tell',
 'by',
 'm',
 't',
 'bank',
 'and',
 'the',
 'amount',
 'that',
 'be',
 'report',
 'to',
 'the',
 'credit',
 'reporting',
 'agency',
 'i',
 'be',
 'advise',
 'to',
 'write',
 'a',
 'good',
 'will',
 'letter',
 'in',
 'order',
 'to',
 'address',
 'the',
 'issue',
 'and',
 'request',
 'the',
 'negative',
 'entry',
 'be',
 'remove',
 'from',
 '-PRON-',
 'credit',
 'report',
 'all',
 'together',
 'i',
 'have',
 'a',
 'vehicle',
 'that',
 'be',
 'steal',
 'and',
 '-PRON-',
 'be',
 'declare',
 'a',
 'total',
 'loss',
 'by',
 'insurance',
 'company',
 'the',
 'insurance',
 'company',
 'and',
 'the',
 'gap',
 'insurancw',
 'companypaid',
 'the',
 'outstanding',
 'balance',
 'of',
 'the',
 'loan',
 'but',
 'i',
 'be',
 'tell',
 'by',
 'm',
 't',
 'bank',
 'that',
 'there',
 'be',
 'still',
 'a',
 'balance',
 'due',
 'on',
 'the',
 'loan',
 'in',
 'good',
 'faith',
 'without',
 'have',
 'receive',
 

In [17]:
# common_terms = ["of", "with", "without", "and", "or", "the", "a","on"]

In [18]:
# phrases = Phrases(sent, min_count=30, common_terms=common_terms,progress_per=10000)

In [19]:
# bigram = Phraser(phrases)

In [20]:
# sentences = bigram[sent]
sentences = sent

Most Frequent Words:

In [21]:
word_freq = defaultdict(int)
for sents in sentences:
    for i in sents:
        word_freq[i] += 1
len(word_freq)

42264

In [22]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]

['-PRON-',
 'the',
 'be',
 'i',
 'to',
 'and',
 'a',
 'have',
 'of',
 'that',
 'in',
 'on',
 'not',
 'for',
 'this',
 'with',
 'do',
 'credit',
 'account',
 'from']

In [23]:
import multiprocessing

from gensim.models import Word2Vec

In [24]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [25]:
w2v_model = Word2Vec(min_count=10,
                     window=10,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [26]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 18:47:59: collecting all words and their counts
INFO - 18:47:59: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:47:59: PROGRESS: at sentence #10000, processed 1803416 words, keeping 15919 word types
INFO - 18:47:59: PROGRESS: at sentence #20000, processed 3551751 words, keeping 22768 word types
INFO - 18:47:59: PROGRESS: at sentence #30000, processed 5326612 words, keeping 28437 word types
INFO - 18:48:00: PROGRESS: at sentence #40000, processed 7134604 words, keeping 33589 word types
INFO - 18:48:00: PROGRESS: at sentence #50000, processed 8937295 words, keeping 37562 word types
INFO - 18:48:00: PROGRESS: at sentence #60000, processed 10713387 words, keeping 40692 word types
INFO - 18:48:01: collected 42264 word types from a corpus of 11741263 raw words and 65552 sentences
INFO - 18:48:01: Loading a fresh vocabulary
INFO - 18:48:01: effective_min_count=10 retains 8204 unique words (19% of original 42264, drops 34060)
INFO - 18:48:01: effective_min_c

Time to build vocab: 0.04 mins


In [27]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=1, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 18:48:01: training model with 3 workers on 8204 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=10
INFO - 18:48:02: EPOCH 1 - PROGRESS: at 6.78% examples, 220008 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:03: EPOCH 1 - PROGRESS: at 13.28% examples, 212123 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:04: EPOCH 1 - PROGRESS: at 19.31% examples, 203712 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:05: EPOCH 1 - PROGRESS: at 26.48% examples, 208599 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:06: EPOCH 1 - PROGRESS: at 33.65% examples, 211100 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:07: EPOCH 1 - PROGRESS: at 40.49% examples, 212550 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:08: EPOCH 1 - PROGRESS: at 47.33% examples, 213556 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:09: EPOCH 1 - PROGRESS: at 54.02% examples, 214213 words/s, in_qsize 5, out_qsize 0
INFO - 18:48:10: EPOCH 1 - PROGRESS: at 60.84% examples, 215000 words/s, in_qsize 5, out_q

Time to train the model: 0.26 mins


In [40]:
# generic word2vec model
w2v_model.wv.save_word2vec_format("cfpb_generic_word2vec.model")

INFO - 19:29:29: storing 8204x300 projection weights into cfpb_generic_word2vec.model


In [43]:
w2v_model.wv.most_similar(positive=["delinquent"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('past', 0.8786083459854126),
 ('delinquency', 0.8634366393089294),
 ('due', 0.85023432970047),
 ('current', 0.8459511399269104),
 ('remain', 0.8451446890830994),
 ('reflect', 0.839966356754303),
 ('full', 0.8087552785873413),
 ('seven', 0.8065049052238464),
 ('unpaid', 0.8056871891021729),
 ('late', 0.8025687336921692),
 ('payment', 0.8020836710929871),
 ('installment', 0.8004365563392639),
 ('although', 0.8001413345336914),
 ('forgive', 0.7789462804794312),
 ('adjust', 0.7732735276222229),
 ('monthly', 0.7648977041244507),
 ('incorrectly', 0.7529599666595459),
 ('which', 0.7499921321868896),
 ('si', 0.7421281337738037),
 ('mark', 0.7374648451805115)]

In [39]:
w2v_model.wv.most_similar(positive=["mod"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('modification', 0.9357266426086426),
 ('caliber', 0.9327195882797241),
 ('underwriter', 0.9135488271713257),
 ('bsi', 0.9116531610488892),
 ('trial', 0.9059758186340332),
 ('flagstar', 0.9036935567855835),
 ('counselor', 0.8907279968261719),
 ('postpone', 0.8878750205039978),
 ('process', 0.8874344825744629),
 ('underwriting', 0.8843114972114563),
 ('permanent', 0.8815696239471436),
 ('hamp', 0.881142258644104),
 ('approval', 0.8747977018356323),
 ('hud', 0.8693662285804749),
 ('finalize', 0.8624701499938965),
 ('modify', 0.8597497940063477),
 ('fha', 0.8594725728034973),
 ('short', 0.8579661250114441),
 ('selene', 0.8526699542999268),
 ('docs', 0.8470214009284973)]

In [32]:
w2v_model.wv.most_similar(positive=["foreclosure"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('foreclose', 0.9448155164718628),
 ('lieu', 0.9432709217071533),
 ('sale', 0.8881224393844604),
 ('caliber', 0.8782910704612732),
 ('citimortgage', 0.8733816146850586),
 ('trustee', 0.8587684035301208),
 ('investor', 0.8553109169006348),
 ('deed', 0.852775514125824),
 ('postpone', 0.8508062958717346),
 ('hud', 0.8505468964576721),
 ('mediation', 0.8452327847480774),
 ('appeal', 0.8379994630813599),
 ('sps', 0.8352752923965454),
 ('motion', 0.8312446475028992),
 ('short', 0.8286450505256653),
 ('grant', 0.8284305334091187),
 ('counselor', 0.8217655420303345),
 ('hearing', 0.8212093114852905),
 ('mod', 0.8187493085861206),
 ('nationstar', 0.8150159120559692)]

In [38]:
len(w2v_model.wv.vocab)

8204